In [9]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [8]:
client = dataiku.api_client()
#prj= client.get_project(dataiku.default_project_key())
#datasets = prj.list_datasets()

#client = DSSClient(host, apiKey)
dss_projects = client.list_project_keys()
print(dss_projects)

['ADMIN_PROJECT', 'BACKEND_BREAKER', 'BASKETBALLv2', 'BASKETBALLv2_2', 'CUSTOMER_CHURN_BROKEN', 'DKU_TSHIRTS', 'DKU_TUTORIAL_BASICS_101', 'GROUP_A_PROJECT', 'GROUP_B_PROJECT', 'THEBIGSLEEP']


In [55]:
for ProjectKey in dss_projects:
    project = client.get_project(ProjectKey)
    datasets = project.list_datasets()
    print("Datasets in Project: ", ProjectKey)
    index = 0
    for dataset in datasets:
        # Quick access to main information in the dataset list item
        print("Name: %s" % dataset.name)
        print("Type: %s" % dataset.type)
        print("Connection: %s" % dataset.connection)
        print("Tags: %s" % dataset.tags) # Returns a list of strings

        # You can also use the list item as a dict of all available dataset information
        print("Raw params: %s" % dataset['params'])
        print("Raw schema: %s" % dataset['schema'])
        
        if (dataset.type == 'PostgreSQL'):
#        if dataset.type == 'Snowflake':
#            print(dataset.name)

            dataset_to_update = project.get_dataset(dataset.name)
#             print("Dataset to update:")
#             print(dataset_to_update)
            
            # Get dataset settings
            settings = dataset_to_update.get_settings()
            # get raw dataset settings, which is editable
            settings_raw = settings.get_raw()
            print("\n")

            print("Project: ", ProjectKey, " Dataset: ", dataset.name, "params:")
            print(settings_raw['params'])
            print("Project", ProjectKey, "Dataset", dataset.name, "params schema:")
#            print(settings_raw['params']['schema'])
            print("<<<<<<<<<<SHUDEBUG0<<<<<<<<<<<")
#            print(settings_raw['params']['catalog'])
            print(settings_raw['params']['table'])
            print(">>>>>>>>>>>>SHUDEBUG1>>>>>>>>>>>")
            print(settings_raw['schema'])
            # pp.pprint(settings_raw['params'])
            if 'schema' in settings_raw['params']:
                print(">>>>>>>>>>>>SHUDEBUG2<<<<<<<<<<<")
                print(settings_raw['params']['schema'])
                
                
#             if 'schema' in settings_raw['params']:
#                 if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
#                     new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
#                     update_check = new_values.empty
#                     if not new_values.empty:
#                         print("Inside If", update_check, new_values,"*****", new_values.index[0])
#                         settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
#                         settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
#                         settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
#                         settings.save()
#                         settings_raw["schema"] = {"columns":[]}
#                         settings.save()
#                         settings = dataset_to_update.autodetect_settings()
#                         settings.save()
#                         new_values = None
#                        index += 1            

Datasets in Project:  ADMIN_PROJECT
Datasets in Project:  BACKEND_BREAKER
Name: bikeshare
Type: UploadedFiles
Connection: None
Tags: []
Raw params: {'uploadConnection': 'Default (in DSS data dir.)', 'notReadyIfEmpty': False, 'filesSelectionRules': {'mode': 'ALL', 'excludeRules': [], 'includeRules': [], 'explicitFiles': []}}
Raw schema: {'columns': [{'name': 'Duration', 'type': 'string'}, {'name': 'Start date', 'type': 'string'}, {'name': 'End date', 'type': 'string'}, {'name': 'Start station number', 'type': 'string'}, {'name': 'Start station', 'type': 'string'}, {'name': 'End station number', 'type': 'string'}, {'name': 'End station', 'type': 'string'}, {'name': 'Bike number', 'type': 'string'}, {'name': 'Member type', 'type': 'string'}], 'userModified': True}
Datasets in Project:  BASKETBALLv2
Name: Basketball_EXP_TM
Type: Filesystem
Connection: filesystem_managed
Tags: []
Raw params: {'connection': 'filesystem_managed', 'path': '${projectKey}/Basketball_EXP_TM', 'notReadyIfEmpty': F



Project:  BASKETBALLv2_2  Dataset:  basketball_avgztot_by_exp params:
{'writeWithCopyBadDataBehavior': 'NOVERIFY_ERROR', 'connection': 'postgres_conn', 'notReadyIfEmpty': False, 'mode': 'table', 'partitioningType': 'custom', 'normalizeDoubles': True, 'table': '${projectKey}_basketball_avgztot_by_exp', 'tableCreationMode': 'auto', 'writeInsertBatchSize': 10000, 'writeJDBCBadDataBehavior': 'DISCARD_ROW', 'readColsWithUnknownTzAsDates': False, 'readSQLDateColsAsDSSDates': True}
Project BASKETBALLv2_2 Dataset basketball_avgztot_by_exp params schema:
<<<<<<<<<<SHUDEBUG0<<<<<<<<<<<
${projectKey}_basketball_avgztot_by_exp
>>>>>>>>>>>>SHUDEBUG1>>>>>>>>>>>
{'columns': [{'name': 'Exp', 'type': 'bigint', 'originalType': 'int8'}, {'name': 'avg', 'type': 'double', 'originalType': 'float8'}], 'userModified': True}
Name: py_basketball_prepared
Type: Filesystem
Connection: filesystem_managed
Tags: []
Raw params: {'connection': 'filesystem_managed', 'path': '${projectKey}/py_basketball_prepared', 'no

Datasets in Project:  GROUP_A_PROJECT
Name: dow_jones_index
Type: UploadedFiles
Connection: None
Tags: []
Raw params: {'uploadConnection': 'Default (in DSS data dir.)', 'notReadyIfEmpty': False, 'filesSelectionRules': {'mode': 'ALL', 'excludeRules': [], 'includeRules': [], 'explicitFiles': []}}
Raw schema: {'columns': [{'name': 'quarter', 'type': 'string'}, {'name': 'stock', 'type': 'string'}, {'name': 'date', 'type': 'string'}, {'name': 'open', 'type': 'string'}, {'name': 'high', 'type': 'string'}, {'name': 'low', 'type': 'string'}, {'name': 'close', 'type': 'string'}, {'name': 'volume', 'type': 'string'}, {'name': 'percent_change_price', 'type': 'string'}, {'name': 'percent_change_volume_over_last_wk', 'type': 'string'}, {'name': 'previous_weeks_volume', 'type': 'string'}, {'name': 'next_weeks_open', 'type': 'string'}, {'name': 'next_weeks_close', 'type': 'string'}, {'name': 'percent_change_next_weeks_price', 'type': 'string'}, {'name': 'days_to_next_dividend', 'type': 'string'}, {'n

In [51]:
# index = 0
# for dataset in datasets:
#     #pp.pprint(dataset)
#     print(dataset)
#     if dataset.type == 'Snowflake':
#         dataset_to_update = prj.get_dataset(dataset.name)
#         # Get dataset settings
#         settings = dataset_to_update.get_settings()
#         # get raw dataset settings, which is editable
#         settings_raw = settings.get_raw()
#         # pp.pprint(settings_raw['params'])
#         if 'schema' in settings_raw['params']:
#             if settings_raw['params']['schema'] in ['AWB_CDW','AWB_CDL', 'AWB_ADHOC','AWB_DATAIKU_CDW', 'AWB_DATAIKU_CDL','AWB_DATAIKU_ADHOC']:
#                 new_values = workflow_Alteration_df[workflow_Alteration_df['Existing_Source'.strip()] == settings_raw['params']['table']]
#                 update_check = new_values.empty
#                 if not new_values.empty:
#                     print("Inside If", update_check, new_values,"*****", new_values.index[0])
#                     settings_raw['params']['catalog'] = new_values['GB_Database'.strip()][new_values.index[0]]
#                     settings_raw['params']['schema'] = new_values['GB_Schema'.strip()][new_values.index[0]]
#                     settings_raw['params']['table'] = new_values['GB_TBL_Name'.strip()][new_values.index[0]]
#                     settings.save()
#                     settings_raw["schema"] = {"columns":[]}
#                     settings.save()
#                     settings = dataset_to_update.autodetect_settings()
#                     settings.save()
#                     new_values = None
#                     index += 1